# Instrucciones de Ejecución

> **IMPORTANTE:** Ejecutar las celdas de arriba hacia abajo sin saltar ningún paso.  
> - Cada sección depende de la salida de la anterior (instalación, carga de datos, transformación, análisis).  
> - Si ejecutas celdas fuera de orden, es posible que falten variables o dependencias y el notebook falle.

---

## Justificación del Dataset

En lugar de descargar los datos directamente desde el portal oficial de la Oficina de Estadísticas de Transporte de EE. UU., hemos utilizado el conjunto de datos de Kaggle (`usdot/flight-delays`).  
- Al intentar acceder a los archivos CSV originales en el sitio de la BTS, no se encontraban disponibles.
- Los datos de Kaggle suelen estar preprocesados y “limpios” (columnas bien definidas, sin valores corruptos ni codificaciones extrañas), lo que facilita la carga en Spark y evita pasos adicionales de limpieza manual.  
- Además, Kaggle garantiza una URL de descarga estable y un único CSV listo para usar.


In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count
from pyspark.sql.types import DoubleType


In [16]:
# 1) Inicia Spark
spark = SparkSession.builder \
    .appName("ProcesamientoVuelos2015") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
# 2) Carga el CSV en un DataFrame de Spark
flights_df = spark.read.csv(
    "data/flights.csv",
    header=True,
    inferSchema=True
)

In [17]:



# 3) Retraso promedio de llegada en un aeropuerto específico
#    Ajustamos a la columna ARRIVAL_DELAY y DESTINATION_AIRPORT
target_airport = "JFK"

avg_delay_df = (
    flights_df
    .filter(
        (col("DESTINATION_AIRPORT") == target_airport) &
        col("ARRIVAL_DELAY").isNotNull()
    )
    .agg(
        avg(col("ARRIVAL_DELAY")).cast(DoubleType()).alias("PromedioArrDelay")
    )
)



In [18]:
print(f"Retraso promedio de llegada en {target_airport}:")
avg_delay_df.show(truncate=False)

Retraso promedio de llegada en JFK:
+-----------------+
|PromedioArrDelay |
+-----------------+
|6.699455162850623|
+-----------------+



In [19]:



# 4) Top-10 rutas más populares (origen → destino)
#    Ahora con ORIGIN_AIRPORT y DESTINATION_AIRPORT
top_routes_df = (
    flights_df
    .groupBy("ORIGIN_AIRPORT", "DESTINATION_AIRPORT")
    .agg(count("*").alias("CantidadVuelos"))
    .orderBy(col("CantidadVuelos").desc())
    .limit(10)
)

print("Top 10 rutas más populares (Origen → Destino):")
top_routes_df.show(truncate=False)


# 5) Cierra la sesión de Spark
spark.stop()


Top 10 rutas más populares (Origen → Destino):


+--------------+-------------------+--------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|CantidadVuelos|
+--------------+-------------------+--------------+
|SFO           |LAX                |13744         |
|LAX           |SFO                |13457         |
|JFK           |LAX                |12016         |
|LAX           |JFK                |12015         |
|LAS           |LAX                |9715          |
|LGA           |ORD                |9639          |
|LAX           |LAS                |9594          |
|ORD           |LGA                |9575          |
|SFO           |JFK                |8440          |
|JFK           |SFO                |8437          |
+--------------+-------------------+--------------+

